In [1]:
# # ユーザー認証
# from google.colab import auth
# auth.authenticate_user()

#Google setting
from google import colab
colab.drive.mount('/content/gdrive')

#Directory setting
b_dir='./gdrive/My Drive/AI_Seminar/NLP_1/' # Setting working directory
dataset_dir=b_dir+'dataset/'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
%%bash

free -h

# ツールのダウンロード
apt-get update
apt-get install mecab file swig libmecab-dev mecab-ipadic-utf8
pip install mecab-python3==0.996.5
pip install transformers==2.11.0
pip install tqdm
pip install category_encoders

              total        used        free      shared  buff/cache   available
Mem:            12G        624M        5.1G        1.0M        7.0G         11G
Swap:            0B          0B          0B
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic I

In [3]:
# insall
import numpy as np
import pandas as pd
import re
import torch
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from scipy import stats
from sklearn.model_selection import KFold
import os,urllib,glob,tarfile
from transformers import BertJapaneseTokenizer, BertModel, BertConfig, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score
import category_encoders as ce

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
# データの読み込み
train_df = pd.read_csv("https://drive.google.com/uc?export=download&id=1mYL2JNkfIPRvvDdw8m0Fnh8mKYb4ND5S")  # train.csv
valid_df = pd.read_csv("https://drive.google.com/uc?export=download&id=1Mdyv3DPcb1AncxE7H739QWyPkMqcdjqI")  # dev.csv
test_df = pd.read_csv("https://drive.google.com/uc?export=download&id=1yHKfDrs6ZAx4fizToTF7GL7ROvGk7Ry5")  # test.csv
# train_df = pd.read_csv("./dataset/train.csv")
# val_df = pd.read_csv("./dataset/dev.csv")
# test_submission = pd.read_csv("./dataset/test.csv")

train_df.head(5)

,movieName,title,text,rating
0,HERO,楽しみにしてるよ～～!!!,この前、HEROを１話から見直しました!!もぅドキドキで楽しいっ!!あたしも、西遊記見たとき...,5
1,インセプション,期待しすぎた…,DVDでの鑑賞。\nちょっと観て時間がたったので、ざっくりレビューです。\n\n根本的なアイ...,3
2,パッチギ！ LOVE&PEACE,真面目に生きている在日の人たちが可哀想,こんな変な映画が、全国上映されるからまた、在日批判が出てくるんじゃないの？,1
3,ダ・ヴィンチ・コード,いまいちの映画でした,はっきり言って期待しすぎたのが悪かったのか全く面白くなかったです。この手の映画は宗教がらみな...,2
4,踊る大捜査線 THE MOVIE 3 ヤツらを解放せよ！,残念なチャリティーマッチ,制作側の意図や事情を知らずにむやみに酷評するのははばかられますが、はっきり言ってつまらなかっ...,2


In [5]:
list_cols = ['movieName']
# ce_ohe = ce.OneHotEncoder(cols=list_cols,handle_unknown='impute')
ce_oe = ce.OrdinalEncoder(cols=list_cols, handle_unknown='impute')
# train_df, valid_df, test_df = ce_ohe.fit_transform([train_df, valid_df, test_df])
train_df = ce_oe.fit_transform(train_df)
valid_df = ce_oe.fit_transform(valid_df)
test_df = ce_oe.fit_transform(test_df)

/usr/local/lib/python3.6/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [6]:
print(train_df)
print(valid_df)

       movieName  ... rating
0              1  ...      5
1              2  ...      3
2              3  ...      1
3              4  ...      2
4              5  ...      2
...          ...  ...    ...
29995         28  ...      1
29996         30  ...      1
29997         11  ...      5
29998         16  ...      1
29999          5  ...      3

[30000 rows x 4 columns]
     movieName  ... rating
0           20  ...      5
1           20  ...      1
2           29  ...      5
3            3  ...      5
4           30  ...      5
..         ...  ...    ...
995         29  ...      2
996         29  ...      5
997         15  ...      4
998         27  ...      2
999         28  ...      5

[1000 rows x 4 columns]


In [7]:
train_df['text'] = train_df['title'].str.cat("\n"+train_df['text'])
valid_df['text'] = valid_df['title'].str.cat("\n"+valid_df['text'])
test_df['text'] = test_df['title'].str.cat("\n"+test_df['text'])

In [8]:
print(train_df["text"])
print(valid_df["text"])

0        楽しみにしてるよ～～!!!\nこの前、HEROを１話から見直しました!!もぅドキドキで楽しい...
1        期待しすぎた…\nDVDでの鑑賞。\nちょっと観て時間がたったので、ざっくりレビューです。\...
2        真面目に生きている在日の人たちが可哀想\nこんな変な映画が、全国上映されるからまた、在日批判...
3        いまいちの映画でした\nはっきり言って期待しすぎたのが悪かったのか全く面白くなかったです。こ...
4        残念なチャリティーマッチ\n制作側の意図や事情を知らずにむやみに酷評するのははばかられますが...
                               ...                        
29995    音楽以外全部沈没\nすべてはキャストから始まった。\nブラッド・ピット、ケイト・ブランシェッ...
29996    冷静に書いています。\n観賞後、１週間が経ちました。\nやっと冷静に書けるようになったので、...
29997    百聞は一見にしかず。\n11月に日本初上陸した、ららぽーと船橋の「dolby ATMOS」に...
29998    作品としては最悪\nまず、作品を観た感想としては、「意味不明」でした。\n基本的に主人公達が...
29999    みなさんの評価が最低だったので\n期待せず観れたことがよかったのかな？\nアクビはでんかんっ...
Name: text, Length: 30000, dtype: object
0      とても良かったと思います。\n非常見やすかった。\nアリスと個々のキャラクターとの演技を含む...
1      別に…\n3D眼鏡は壊れ(傷がつい)ていると1秒ごとに画面が緑になる。\n…というアクシデン...
2      良い映画に出会えた\n酷評が多かったので期待半分でしたが、とても高揚して映画館を出てきました...
3      歴史を受け止めた、素晴らしい作品☆\n前作も泣けて熱くなりましたが、今回はより熱くなる作品で...
4      ワタクシの考えを変えさせられた作品。\n原作が好きなので視聴してみました。\n\nアクション...
                         

In [9]:
train_df['movieName'] = train_df['movieName'].astype(str)
valid_df['movieName'] = train_df['movieName'].astype(str)
test_df['movieName'] = train_df['movieName'].astype(str)

In [10]:
train_df['text'] = train_df['movieName'].str.cat("\n"+train_df['text'])
valid_df['text'] = valid_df['movieName'].str.cat("\n"+valid_df['text'])
test_df['text'] = test_df['movieName'].str.cat("\n"+test_df['text'])

In [11]:
# tokenizerの定義とvocabを作成
tokenizer = BertJapaneseTokenizer.from_pretrained("cl-tohoku/bert-base-japanese")
pad = tokenizer.vocab["[PAD]"]
max_lengths=512

In [12]:
# 上記のinput_dataの作成方法をもとにdataset化
class ReviewDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels=[]):
        self.input_ids, self.segment_ids, self.attention_masks = [],[],[]
        for text in tqdm(texts):
            token = ["[CLS]"]+tokenizer.tokenize(text)[:max_lengths-2]+["[SEP]"]
            input_id = tokenizer.convert_tokens_to_ids(token)
            segment_id = [0]*max_lengths
            attention_mask = [1]*len(input_id)+[0]*(max_lengths - len(input_id))
            input_id = input_id+[pad]*(max_lengths-len(input_id))
            self.input_ids.append(input_id)
            self.segment_ids.append(segment_id)
            self.attention_masks.append(attention_mask)
        self.input_ids = np.array(self.input_ids)
        self.segment_ids = np.array(self.segment_ids)
        self.attention_masks = np.array(self.attention_masks)
        self.labels = labels

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        if len(self.labels):
            return self.input_ids[idx], self.segment_ids[idx], self.attention_masks[idx], self.labels[idx]
        else:
            return self.input_ids[idx], self.segment_ids[idx], self.attention_masks[idx]

In [13]:
# datasetとdataloaderの作成
batch_size = 8  # 8
# X, y = train_df.drop("rating").values, train_df["rating"].values-1
X, y = train_df["text"].values, train_df["rating"].values-1
# X, y = train_df["title"].values, train_df[["rating_1", "rating_2", "rating_3", "rating_4", "rating_5"]].values
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=100,stratify=y)
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=100,stratify=y)
train_ds = ReviewDataset(texts=X_train, labels=y_train)
train_dl = torch.utils.data.DataLoader(
    train_ds, batch_size=batch_size, shuffle=True)
X_validOrg, y_validOrg = valid_df["text"].values, valid_df["rating"].values-1
X_valid, y_valid = np.concatenate([X_val, X_validOrg]), np.concatenate([y_val, y_validOrg])
X_valid, _, y_valid, _ = train_test_split(X_valid, y_valid, test_size=0.64, random_state=100, stratify=y_valid)
valid_ds = ReviewDataset(texts=X_valid, labels=y_valid)
valid_dl = torch.utils.data.DataLoader(
    valid_ds, batch_size=batch_size, shuffle=False)

In [14]:
print(X_valid)
print(y_valid)

['14\nええやん\n自分みたいにcmしか見ていない人にはゴジラが世界を滅ぼすように思ったんじゃないでしょうか。\n全体を通して怪獣同士のバトルが少なかったのは残念ですが、それでもゴジラの咆哮は迫力満点でした。\n\nこんなのゴジラじゃないと思う人も多いでしょうが日本のゴジラ知ってようが知らまいが楽しめる作品だと思います。'
 '7\nナタリーポートマンの演技に釘付け。\n作品自体は重苦しい雰囲気でした。\n主人公の緊迫感が、映画をみている側にも\n伝わってきました。何かに追われる感覚です。\n\nナタリーポートマンの迫真の演技から、終始目が話せませんでした。'
 '14\n普通に面白いパニック映画です\nゴジラ映画ファンとしての過剰な期待は禁物。アメリカ映画なのでCG駆使したモンスター映画としてはスリル満点で楽しめました。\n不満な点としてはゴジラよりもギャオスの出来損ない怪獣の方が目立っている、\nせっかく渡辺謙をわざわざ芹沢なんて名乗らせて登場させているのに全く生かされていない、\nまあ普通でしょう、PART２はないな。\nクレジットの最後に円谷英二、田中友幸、本多猪四郎にささぐなんて出たらブーイングもんでしたが、それはなかったです。'
 ... '3\nダニ映画\nダニだ'
 '11\nなんで「ゼロ」つけた？。\n原題は「GRAVITY」。\nあのラストシーンの後にタイトルが出て、なるほど！と思わせる仕掛けがあるのに。\n「ゼロ」つけちゃったら、なんだか意味が違ってくる気がするのですが・・・。\nまぁいいや、この際「ゼロ」は無視しましょう。\n\nサンドラ・ブロックはすごい女優だ。\nいつものサンドラらしさも所々覗かせつつ、新しい顔を見せてくれた。\n最後に重力をかみしめた後の、なんとも凛々しい表情。\n素晴らしかったです。\n\n映像が凄いのは、もう口コミ通り。\nこれは映画館の大スクリーンで３Ｄで観なければ意味が無い。\n３Ｄ好きな私としては、\nこれをきっかけに、日本ではイマイチ浸透力の薄い３Ｄに、\n新たな可能性が生まれてくれたらと切に願う。\n\n映像、音楽、俳優、そしてストーリー。\nすべて一体となって、とても素晴らしい映画。\nこれは文句無く、本年度ナンバーワンの作品。'
 '28\n愚かな生き物\n素晴らしい上質の映像美、奥行き深く難解な（

In [15]:
# Bertの事前学習済みエンコーダーを使用して5値クラス分類器を作成

class BertClassification(nn.Module):
    def __init__(self, bert):
        super(BertClassification, self).__init__()

        # BERTモジュール
        self.bert = bert  # BERTモデル

        self.cls = nn.Linear(in_features=768, out_features=5)

        # 重み初期化処理
        nn.init.normal_(self.cls.weight, std=0.02)
        nn.init.normal_(self.cls.bias, 0)

    def forward(self, input_ids, token_type_ids=None, attention_mask=None):

        # BERTの基本モデル部分の順伝搬
        # 順伝搬させる
        pooled_output = self.bert(
                  input_ids, token_type_ids, attention_mask)[1]

        # 入力文章の1単語目[CLS]の特徴量を使用して、ポジ・ネガを分類します
        pooled_output = pooled_output.view(-1, 768)  # sizeを[batch_size, hidden_size]に変換
        outputs = self.cls(pooled_output)

        return outputs


In [16]:
cuda = torch.cuda.is_available()
# 学習済みモデルをダウンロード
bert = BertModel.from_pretrained("cl-tohoku/bert-base-japanese")
model = BertClassification(bert)
# 高速化
torch.backends.cudnn.benchmark = True
if cuda:
    model.cuda()
# optimizerの設定
# optimizer = optim.Adam(model.parameters(),lr = 4e-4 ,betas=(0.9, 0.999))
# optimizer = optim.AdamW(model.parameters(),lr = 1e-5 ,betas=(0.9, 0.999))
optimizer = optim.AdamW(model.parameters(), lr = 1e-4)
# scheduler = get_linear_schedule_with_warmup(
#     optimizer,
#     num_warmup_steps=(len(X_train) // batch_size + 1) * 1,
#     num_training_steps=(len(X_train) // batch_size + 1) * 5
# )

# 損失関数の設定
criterion = nn.CrossEntropyLoss()

In [17]:
# # 最後以外のBertLayerモジュールのパラメータを固定(今回は速さのためにしているだけ。精度を上げるなら固定せずに実行するべき)
# for param in model.bert.encoder.layer[:-1].parameters():
#     param.requires_grad = False

In [18]:
# model.load_state_dict(torch.load('./dataset/model.hdf5'))

In [19]:
## 学習します。
epochs = 7
# best_train_acc = 0.0
# best_val_acc = 0.0
best_train_mae = np.inf
best_valid_mae = np.inf
for epoch in range(epochs):
    total_loss = 0.0
    accuracy = 0.0
    mae = 0.0
    model.train()

    print("epoch {} start!".format(epoch+1))
    # train
    for iter_num, (input_ids, segment_ids, attention_masks, labels) in tqdm(enumerate(train_dl), total=len(train_dl)):
        optimizer.zero_grad()
        if cuda:
            input_ids, segment_ids, attention_masks, labels =\
            input_ids.cuda(), segment_ids.cuda(), attention_masks.cuda(), labels.cuda()
        # forward(順伝搬)
        outputs = model(input_ids = input_ids,
                        token_type_ids = segment_ids,
                        attention_mask = attention_masks)
        pred_proba = outputs.softmax(dim=1)
        pred = torch.argmax(pred_proba, axis=1)
        # pred = torch.max(pred_proba, dim=1)
        loss = criterion(outputs, labels)

        # backward(逆伝搬)
        loss.backward()
        optimizer.step()
        # scheduler.step()

        total_loss += loss.item()
        accuracy += (pred==labels).sum().item()
        mae += torch.abs(pred - labels).sum().item()
        # accuracy += (pred.indices==labels).sum().item()
        # mae += torch.abs(pred.indices - labels).sum().item()
        #50 iterごとにlossとaccuracyを表示
        if (iter_num+1) % 50 == 0:
            now_size = batch_size*(iter_num+1)
            print("{} iter loss:{:.4f} accuracy:{:.4f} MAE:{:.4f}".format(
            iter_num+1,total_loss/(iter_num+1),accuracy/now_size,mae/now_size))
            # if best_train_acc < accuracy/now_size:
            #     torch.save(model.state_dict(), './dataset/model.hdf5')
            #     best_train_acc = accuracy/now_size
            if best_train_mae > mae/now_size:
                torch.save(model.state_dict(), dataset_dir+'model_train_allin.hdf5')
                best_train_mae = mae/now_size

    total_loss /= len(train_dl)
    accuracy /= len(train_ds)
    mae /= len(train_ds)

    # valididation
    valid_total_loss = 0.0
    valid_accuracy = 0.0
    valid_mae = 0.0
    model.eval()
    for input_ids, segment_ids, attention_masks, labels in tqdm(valid_dl):
        if cuda:
            input_ids, segment_ids, attention_masks, labels =\
            input_ids.cuda(), segment_ids.cuda(), attention_masks.cuda(), labels.cuda()
        with torch.no_grad():
            outputs = model(input_ids = input_ids,
                        token_type_ids = segment_ids,
                        attention_mask = attention_masks)
        pred_proba = outputs.softmax(dim=1)
        pred = torch.argmax(pred_proba, axis=1)
        # pred = torch.max(pred_proba, dim=1)
        loss = criterion(outputs, labels)
        valid_total_loss += loss.item()
        valid_accuracy += (pred==labels).sum().item()
        valid_mae += torch.abs(pred - labels).sum().item()
        # valid_accuracy += (pred.indices==labels).sum().item()

    valid_total_loss /= len(valid_dl)
    valid_accuracy /= len(valid_ds)
    valid_mae /= len(valid_ds)
    print("epoch:{} total loss:{:.4f}, accuracy:{:.4f}, MAE:{:.4f}, valid_total loss:{:.4f}, valid_accuracy:{:.4f}, valid_MAE:{:.4f}"\
        .format(epoch+1,total_loss,accuracy,mae,valid_total_loss,valid_accuracy,valid_mae))
    # if best_val_acc < val_accuracy:
    #     torch.save(model.state_dict(), dataset_dir+'model.hdf5')
    #     best_val_acc = val_accuracy
    if best_valid_mae > valid_mae:
        torch.save(model.state_dict(), dataset_dir+'model_valid_allin.hdf5')
        best_valid_mae = valid_mae
# torch.save(model.state_dict(), dataset_dir+'model.hdf5')

epoch 1 start!


50 iter loss:1.6792 accuracy:0.2450 MAE:1.6425
100 iter loss:1.6701 accuracy:0.2150 MAE:1.6838


KeyboardInterrupt: ignored

In [ ]:
model.load_state_dict(torch.load(dataset_dir+'model_valid_allin.hdf5'))

In [ ]:
# test datset dataloaderを作成。
test_ds = ReviewDataset(texts=test_df["text"].values)
test_dl = torch.utils.data.DataLoader(
    test_ds, batch_size=1, shuffle=False)

In [ ]:
if cuda:
    model.cuda()
else:
    model.cpu()

# 予測を付与。
lists = []
for input_ids, segment_ids, attention_masks in tqdm(test_dl):
    if cuda:
        input_ids, segment_ids, attention_masks =\
        input_ids.cuda(), segment_ids.cuda(), attention_masks.cuda()
    outputs = model(input_ids = input_ids,
                  token_type_ids = segment_ids,
                  attention_mask = attention_masks)
    pred_proba = outputs.softmax(dim=1)
    pred = torch.argmax(pred_proba, axis=1)
    lists += list(pred.cpu().detach().numpy()+1) 

In [ ]:
print(lists)

In [ ]:
# test_df["Label"]=lists
# submission_df=test_df[["Id","Label"]]
# submission_df.to_csv("sample_submission.csv",index=False)

In [ ]:
result = "\n".join([str(rating) for rating in lists])
open(dataset_dir+"test_pred_bert_allin.txt", "w").write(result)